In [1]:
# conda env: pyg (Python3.9.16)
import sys
from datacat4ml.const import DATA_DIR, FIG_DIR, FETCH_DATA_DIR , FETCH_FIG_DIR
from datacat4ml.const import OR_chembl_ids, OR_uniprot_ids
from datacat4ml.utils import mkdirs

import os
from typing import List
import string
# load ipython-sql, a Jupyter Notebook magic extension. 
%load_ext sql 
import sqlite3
from sqlite3 import connect
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.metrics import r2_score,cohen_kappa_score,balanced_accuracy_score,\
    median_absolute_error,matthews_corrcoef
import hashlib
from collections import Counter, defaultdict

from rdkit import Chem
from rdkit.Chem import AllChem

# Connect to the database

In [2]:
%sql sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db

conn = sqlite3.connect('/storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db')

# Fetch GPCR data

## Read Uniprot IDs for GPCR human

In [3]:
# All GPCR proteins in human
GPCR_human = pd.read_csv(os.path.join(DATA_DIR, 'GPCR_human.tsv'),sep='\t')
gpcr_uniprot_ids_list = GPCR_human['Entry'].tolist()
print(f'The number of proteins in GPCR_human is {len(gpcr_uniprot_ids_list)}')
print(f'The first 5 proteins in GPCR_human are {gpcr_uniprot_ids_list[:5]}')

The number of proteins in GPCR_human is 898
The first 5 proteins in GPCR_human are ['Q16570', 'O00590', 'P25106', 'Q9NPB9', 'P18509']


## Write the below columns to a csv file

x
- 'canonical_smiles'
- assays.
    - assay_id,
    - chembl_id --> 'assay_chembl_id',
    - doc_id, 
    
    in the maxMatch:
    - **assay_type**, e.g. B, F.
    - **assay_category**, (very sparse), e.g. Selectivity assay, Affinity biochemical assay, Affinity on-target cellular assay, Affinity phenotypic cellular assay, GPCR beta-arrestin assay, Thermal shift assay.
    - **assay_organism**, e.g. Homo sapiens, Rattus norvegicus
    - **assay_tax_id**, e.g. 9606 (for human), 1280 (for Staphylococcus aureus)
    - **assay_strain**, (median sparse), e.g. LV9, BaL.
    - **assay_tissue**, (median sparse), e.g. Brain, Plasma
    - **assay_cell_type**, (median sparse), e.g. PC-3M, CHO
    - **assay_subcellular_fraction**, (very sparse), e.g. Membrane, Microsome, Mitochondria
    - **bao_format**, (e.g, BAO_0000019: could be cell-based format, organism-based format)
    - **variant_id**, (very sparse)

    something else
    - **assay_test_type**, (sparse), e.g. in vitro, in vivo.
    - **description**
    - cell_id, (median sparse)
    - tissue_id, (median sparse)
    - curated_by, e.g. Autocuration, Intermediate, Expert. 
    - relationship_type, ? e.g. H, U, N, D, M, S 
    - aidx,  ? e.g. CLD0, 1480429, 1480486

support
- target_dictionary.chembl_id --> 'target_chembl_id'
- standard_type, standard_relation, standard_units
'compound_chembl_id'
- assays.relationship_type (e.g. D, stands for direct protein target assigned)

annotation
- asaays.src_id (e.g. integers that are unique identifiers for sources in UniChem )


y
- 'pchembl_value'

# Functions

In [4]:
def gather_data_for_size(uniprot_id: List[str], standard_type:str,
                         onlyDocs=True,removeMutants=True,confidenceScore=8,):
    """ Get the data for the size of the dataset """
    
    if isinstance(uniprot_id, str):
        uniprot_id = [uniprot_id]
    uniprot_id_str = ",".join(f"'{id}'" for id in uniprot_id)

    print('table temp_targets')
    %sql \
        drop table if exists temp_targets
    %sql \
        create table temp_targets as \
        select tid, component_id, accession, target_type, pref_name target_pref_name, chembl_id target_chembl_id \
        from target_components \
        join component_sequences using(component_id) \
        join target_dictionary using(tid) \
        where accession in ({uniprot_id_str}) \
        and target_type = 'SINGLE PROTEIN';
    print(f"The shape of temp_targets is {pd.read_sql('select * from temp_targets', con=conn).shape}")

    # collect activities for the assays that meet the basic criteria
    print('table temp_assays')
    %sql \
        drop table if exists temp_assays
    %sql \
        create table temp_assays as \
        select assay_id, assays.chembl_id assay_chembl_id, assays.description assay_desc, assays.doc_id assay_doc_id, variant_id, \
        docs.year doc_date, docs.chembl_id doc_chembl_id, \
        tid, accession, target_type, target_pref_name, target_chembl_id, \
        count(distinct(molregno)) cnt \
        from activities \
        join assays using(assay_id) \
        join docs on (assays.doc_id = docs.doc_id) \
        join temp_targets using (tid) \
        where standard_type =:standard_type \
        group by tid, assay_doc_id, assay_id \
        order by cnt desc;
    print(f"The shape of temp_assays is {pd.read_sql('select * from temp_assays', con=conn).shape}")

    # now remove rows for assays which are not compatible without curation steps
    print('check onlyDocs')
    if onlyDocs:
        %sql \
            delete from temp_assays where doc_date is null;

    print('check removeMutants')
    if removeMutants:
        %sql \
            delete from temp_assays where variant_id is not null or lower(assay_desc) like '%mutant%'\
                or lower(assay_desc) like '%mutantion%' or lower(assay_desc) like '%variant%';
                
    print('check confidenceScore')
    # filter out assays with confidence score less than confidenceScore
    %sql \
        drop table if exists temp_assays_filtered;
    %sql \
        create table temp_assays_filtered as \
        select ta.*, confidence_score from temp_assays ta \
            join assays using(assay_id) \
            where confidence_score >= :confidenceScore;  
    print(f"The shape of temp_assays_filtered is {pd.read_sql('select * from temp_assays_filtered', con=conn).shape}")
    
    # select activities from the assays we've identified which have between less than maxAssaySize tested compounds --> delete this condition because we are creating a benchmark dataset
    # why are 'pchembl_value' and 'standard_type' used here again? because here join table 'activities' again only on 'assay_id'.
    print('temp_acts')
    %sql \
        drop table if exists temp_acts
    %sql \
        create table temp_acts as \
        select assay_id, assay_chembl_id, tid, target_chembl_id,molregno,doc_chembl_id,standard_type,activity_id \
        from activities \
        join temp_assays_filtered using (assay_id)\
        where standard_type =:standard_type  
    print(f"The shape of temp_acts is {pd.read_sql('select * from temp_acts', con=conn).shape}")

    # count the number of unique compounds in each assay_id
    # `group by`statemenet groups rows that have the same values into summary rows
    print('temp_acts_counts')
    %sql \
        drop table if exists temp_acts_counts
    %sql \
        create table temp_acts_counts as \
        select assay_id, assay_chembl_id, ta.tid, target_chembl_id, count(distinct molregno) cnt, doc_chembl_id \
        from temp_acts ta \
        join assays using (assay_id) \
        group by assay_chembl_id, target_chembl_id, doc_chembl_id \
        order by cnt desc;
    print(f"The shape of temp_acts_counts is {pd.read_sql('select * from temp_acts_counts', con=conn).shape}")
    
    # count the number of unique assays for each target
    print('temp_targets_counts')
    %sql \
        drop table if exists temp_targets_counts
    %sql \
        create table temp_targets_counts as \
        select tid, target_chembl_id, count(distinct assay_chembl_id) target_assay_count, sum(cnt) target_compound_count \
        from temp_acts_counts tac \
        group by target_chembl_id \
        order by target_assay_count desc;
    print(f"The shape of temp_targets_counts is {pd.read_sql('select * from temp_targets_counts', con=conn).shape}")

def append_assay_info(readout = 'Ki',uniprot_id = gpcr_uniprot_ids_list,
                      onlyDocs=True,removeMutants=True,confidenceScore=8):
    '''
    Append the assay information to the output of function gather_data_for_size
    '''
    
    gather_data_for_size(uniprot_id=uniprot_id,standard_type=readout,
                         onlyDocs=onlyDocs,removeMutants=removeMutants,confidenceScore=confidenceScore)

    print (f'raw_data for {readout}')
    
    %sql \
        drop table if exists raw_data
    raw_data = %sql \
        select assay_id, ta_1.assay_chembl_id, ta_1.tid, ta_1.target_chembl_id, standard_type, standard_relation, standard_value, standard_units, pchembl_value, \
        assay_type,assay_category,assay_organism, assay_tax_id, assay_strain, assay_tissue, assay_cell_type, assay_subcellular_fraction, bao_format, ta_1.variant_id, assay_test_type, \
        assay_desc, cell_id, tissue_id, curated_by, relationship_type, aidx, \
        ta_1.confidence_score, molregno, cid.chembl_id compound_chembl_id,canonical_smiles \
        from activities \
        join temp_assays_filtered ta_1 using (assay_id) \
        join temp_targets_counts using (tid) \
        join temp_acts_counts using (assay_id) \
        join assays using (assay_id) \
        join chembl_id_lookup cid on (entity_type='COMPOUND' and molregno=entity_id) \
        join compound_structures using (molregno) \
        where standard_type=:readout;

    return raw_data
    
def hash_assay_info(raw_data, sel_cols:List[str]=['assay_type', 'assay_organism', 'assay_category',
                                                    'assay_tax_id','assay_strain','assay_tissue',
                                                    'assay_cell_type','assay_subcellular_fraction',
                                                    'bao_format', 'variant_id']):
    '''
    For the retrieved data from the database, concatenate the columns related to assay info, hash these values, and convert the data into a pandas dataframe.

    params:
        raw_data: The retrieved data from the database.
        sel_cols: A list of column names to be used for hashing.

    Returns:
        A pandas DataFrame with the original data and an additional 'assay_info_hash' column.

    The string in the sel_cols can be the below:
    'assay_type','assay_organism','assay_category',
    'assay_tax_id','assay_strain','assay_tissue',
    'assay_cell_type','assay_subcellular_fraction','bao_format','variant_id'
    '''

    processed_data = []
    for row in raw_data:
        original_data= [str(value) for value in row]
        # concatenate certain fields and hash
        assay_fields = ''.join([str(row[col]) for col in sel_cols])  # Add all fields you need
        hash_result = hashlib.md5(assay_fields.encode()).hexdigest()
        original_data.append(hash_result)
        processed_data.append(tuple(original_data))

    print(f'The length of the processed_data is {len(processed_data)}')

    return processed_data

## IC50

In [5]:
readout = 'IC50'
print(f'raw_data for {readout}_maxcuration')
raw_data = append_assay_info(readout=readout, uniprot_id=gpcr_uniprot_ids_list,
                             onlyDocs=True,removeMutants=True,confidenceScore=8)
ic50_maxcur_8_data = hash_assay_info(raw_data)

raw_data for IC50_maxcuration
table temp_targets
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
The shape of temp_targets is (381, 6)
table temp_assays
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
The shape of temp_assays is (11432, 13)
check onlyDocs
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
158 rows affected.
check removeMutants
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
266 rows affected.
check confidenceScore
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/

## Ki

In [6]:
readout = 'Ki'
print(f'raw_data for {readout}_maxcuration')
raw_data = append_assay_info(readout=readout, uniprot_id=gpcr_uniprot_ids_list,
                             onlyDocs=True,removeMutants=True,confidenceScore=8)
ki_maxcur_8_data = hash_assay_info(raw_data)

raw_data for Ki_maxcuration
table temp_targets
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
The shape of temp_targets is (381, 6)
table temp_assays
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
The shape of temp_assays is (16610, 13)
check onlyDocs
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
75 rows affected.
check removeMutants
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
193 rows affected.
check confidenceScore
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/che

## EC50

In [7]:
readout = 'EC50'
print(f'raw_data for {readout}_maxcuration')
raw_data = append_assay_info(readout=readout, uniprot_id=gpcr_uniprot_ids_list,
                             onlyDocs=True,removeMutants=True,confidenceScore=8)
ec50_maxcur_8_data = hash_assay_info(raw_data)

raw_data for EC50_maxcuration
table temp_targets
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
The shape of temp_targets is (381, 6)
table temp_assays
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
The shape of temp_assays is (7080, 13)
check onlyDocs
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
114 rows affected.
check removeMutants
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
241 rows affected.
check confidenceScore
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/c

# Write data

### GPCR_type

In [8]:
def write_data(data, filename:str, columns=['assay_id', 'assay_chembl_id', 'tid', 'target_chembl_id', 'standard_type', 'standard_relation', 'standard_value', 'standard_units', 'pchembl_value', 
                                   'assay_type', 'assay_category', # checkpoint 1
                                   'assay_organism', 'assay_tax_id', 'assay_strain', 'assay_tissue', 'assay_cell_type', 'assay_subcellular_fraction', 'bao_format', 'variant_id', 'assay_test_type', 
                                   'assay_desc', 'cell_id', 'tissue_id', 'curated_by', 'relationship_type', 'aidx', 
                                   'confidence_score', 'molregno', 'compound_chembl_id', 'canonical_smiles', 'assay_info_hash']):
    """Save the data to a pandas dataframe"""

    data_df = pd.DataFrame(data, columns=columns)
    
    # write the data to a csv file
    data_df.to_csv(f'{FETCH_DATA_DIR}/{filename}.csv',index=False)
    
    return data_df

In [9]:
ic50_maxcur_df = write_data(ic50_maxcur_8_data, 'IC50_gpcr_maxcur_8_data')
ki_maxcur_df = write_data(ki_maxcur_8_data, 'Ki_gpcr_maxcur_8_data')
ec50_maxcur_df = write_data(ec50_maxcur_8_data, 'EC50_gpcr_maxcur_8_data')

### datapoints with unique 'assay_hash_info'

In [10]:
# only for dataset with maxcuration
def write_data_uniq_cond(df, min_dataset_size=100, readout='IC50', curation='maxcur'):
    """
    write the datasets with unique combinations of 'target_chembl_id' and 'assay_info_hash'
    """
    yaml=[]
    
    collected_data = defaultdict(list)
    assay_cond_accum = defaultdict(list)
    assay_accum = defaultdict(set)

    for i, row in df.iterrows():
        assay_info_hash = row['assay_info_hash']
        target_chembl_id = row['target_chembl_id']
        assay_chembl_id = row['assay_chembl_id']

        # append all columns to the collected_data
        collected_data[(target_chembl_id, assay_info_hash)].append(row.values)
        assay_cond_accum[target_chembl_id].append(assay_info_hash)
        assay_accum[(target_chembl_id,assay_info_hash)].add(assay_chembl_id)

    sets_to_keep = [k for k, v in collected_data.items() if len(v) >= min_dataset_size]
    for k in assay_cond_accum:
        assay_cond_accum[k] = [y for y in assay_cond_accum[k] if (k, y) in sets_to_keep]

    for target_chembl_id, conds in collected_data.keys():
        if (target_chembl_id, conds) not in sets_to_keep:
            continue
        rows = collected_data[(target_chembl_id, conds)]
        cols = np.array(rows).transpose()
        cond_idx = assay_cond_accum[target_chembl_id].index(conds)+1

        columns = df.columns
        df = pd.DataFrame(dict(zip(columns, cols)))
        fname = f'{readout}_target_{target_chembl_id}_{cond_idx}.csv'
        filepath = os.path.join(FETCH_DATA_DIR, f'{readout}_{curation}')
        mkdirs(filepath)
        df.to_csv(os.path.join(filepath, fname), index=False)

        #minAct = min(df.pchembl_value)
        #maxAct = max(df.pchembl_value)
        #medAct = np.median(pd.to_numeric(df.pchembl_value, errors='coerce'))
        actType = readout
        targetd = %sql \
        select * from target_dictionary  \
            where chembl_id=:target_chembl_id
        targetd = dict(targetd[0])    
        template=f'''  {target_chembl_id}-{cond_idx}:
        description: "Target {target_chembl_id}: {targetd['pref_name']}"
        args:
        filename: '{{{{ CATALOG_DIR }}}}/source_data/{fname}'
        smilesColumn: canonical_smiles
        metadata:
        url: https://www.ebi.ac.uk/chembl/target_report_card/{target_chembl_id}/
        source: ChEMBL_32
        target_organism: {targetd['organism']}
        assays_included: {','.join(assay_accum[(target_chembl_id,conds)])}
        num_points: {len(rows)}
        activity_type: {actType}
        driver: intake_rdkit.smiles.SmilesSource
        '''
        #min pchembl_value: {float(minAct):.2f}
        #max pchembl_value: {float(maxAct):.2f}
        #median pchembl_value: {float(medAct):.2f}
        yaml.append(template)
    print('\n'.join(yaml))
    with open(os.path.join(FETCH_DATA_DIR, f'{readout}_{curation}', f'{readout}_{curation}_datasets.yaml'),'w+') as outf:
        header=f'''metadata:

    summary: |
        Collection of {readout} datasets with pchembl_values for bioactivity prediction.
        
    sources:'''
        print(header,file=outf)
        print('\n'.join(yaml),file=outf)

In [11]:
write_data_uniq_cond(ic50_maxcur_df, min_dataset_size=100, readout='IC50', curation='maxcur')
write_data_uniq_cond(ki_maxcur_df, min_dataset_size=100, readout='Ki', curation='maxcur')
write_data_uniq_cond(ec50_maxcur_df, min_dataset_size=100, readout='EC50', curation='maxcur')

 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34.db
Done.
 * sqlite:////storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/chembl_34_sqlite/chembl_34